In [1]:
import pandas as pd

# Load your datasets
dataset1 = pd.read_csv('synthetic_facility_v3.csv')
dataset2 = pd.read_csv('synthetic_hdss_v3.csv')

In [2]:
# Drop the 'id' column in he hdss dataset cause it is null throughout
dataset2 = dataset2.drop(columns=['nationalid'], errors='ignore')

# The dates are actually fine, i realised they are different people and they all have the same format
# Maybe we can work on first trying to match the patient names in the different databases

#The petnames are okay cause they belong to different people



In [3]:

# Assuming the patient name columns are named 'patient_name' in both datasets

# Convert patient names to lowercase for case-insensitive matching
dataset1['firstname'] = dataset1['firstname'].str.lower()
dataset2['firstname'] = dataset2['firstname'].str.lower()

# Find matching patient names
matching_names = pd.merge(dataset1, dataset2, on='firstname', how='inner')

# Save the matching patient names to a new CSV file
matching_names.to_csv('matching_patient_names.csv', index=False)

print("Matching patient names saved to matching_patient_names.csv")


Matching patient names saved to matching_patient_names.csv


In [4]:
#Matching both firstname and last name

# Concatenate last and first names into a single column for both datasets
dataset1['full_name'] = dataset1['lastname'] + ' ' + dataset1['firstname']
dataset2['full_name'] = dataset2['lastname'] + ' ' + dataset2['firstname']


# Convert full names to lowercase for case-insensitive matching
dataset1['full_name'] = dataset1['full_name'].str.lower()
dataset2['full_name'] = dataset2['full_name'].str.lower()


# Find matching full names
matching_names = pd.merge(dataset1, dataset2, on='full_name', how='inner')

# Save the matching full names to a new CSV file
matching_names.to_csv('matching_full_names2.csv', index=False)

print("Matching full names saved to matching_full_names.csv")


Matching full names saved to matching_full_names.csv


In [5]:
from splink.datasets import splink_datasets
import altair as alt

dataset1.head(5)
dataset2.head(5)

ModuleNotFoundError: No module named 'splink'

In [ ]:
# Initialise the linker, passing in the input dataset(s)
from splink.duckdb.linker import DuckDBLinker
linker = DuckDBLinker(dataset1)

In [ ]:
# Initialise the linker, passing in the input dataset(s)
from splink.duckdb.linker import DuckDBLinker
linker = DuckDBLinker(dataset2)

In [ ]:
linker.missingness_chart()

In [ ]:
linker.profile_columns(top_n=10, bottom_n=5)

In [ ]:
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
settings = {"link_type": "dedupe_only"}
linker = DuckDBLinker(df, settings)

blocking_rule_1 = block_on(["substr(first_name, 1,1)", "surname"])
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_1)
print(f"Number of comparisons generated by '{blocking_rule_1.blocking_rule_sql}': {count:,.0f}")

blocking_rule_2 = block_on("surname")
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_2)
print(f"Number of comparisons generated by '{blocking_rule_2.blocking_rule_sql}': {count:,.0f}")

blocking_rule_3 = block_on("email")
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_3)
print(f"Number of comparisons generated by '{blocking_rule_3.blocking_rule_sql}': {count:,.0f}")

blocking_rule_4 = block_on(["city", "first_name"])
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_4)
print(f"Number of comparisons generated by '{blocking_rule_4.blocking_rule_sql}': {count:,.0f}")

In [ ]:
from splink.duckdb.blocking_rule_library import block_on

settings = {
    "link_type": "dedupe_only",
    "comparisons": [
        ctl.name_comparison("first_name"),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],
    "blocking_rules_to_generate_predictions": [
        block_on("first_name"),
        block_on("surname"),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

linker = DuckDBLinker(df, settings)

In [ ]:
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) &lt;= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) &lt;= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) &lt;= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

In [ ]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

In [ ]:
training_blocking_rule = block_on(["first_name", "surname"])
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

In [ ]:
from numpy import fix


training_blocking_rule = block_on("dob")
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

In [ ]:
linker.match_weights_chart()

In [ ]:
settings = linker.save_model_to_json("../demo_settings/saved_model_from_demo.json", overwrite=True)